In [1]:
! pip install transformers pandas torch sentencepiece protobuf

In [2]:
import pandas as pd

In [3]:
# Read the dataset
df = pd.read_csv("heart_disease.csv")

In [4]:
# convert the dataset rows to natural language text descriptions
# age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
def create_descriptions(df):
    descriptions = []
    for _, row in df.iterrows():
        description = (f"Patient with age {row['age']}, sex {row['sex']}, chest pain type {row['cp']}, "
                       f"resting blood pressure {row['trestbps']}, cholesterol level {row['chol']}, "
                       f"fasting blood sugar {row['fbs']}, resting electrocardiographic results {row['restecg']}, "
                       f"maximum heart rate achieved {row['thalach']}, exercise-induced angina {row['exang']}, "
                       f"ST depression induced by exercise {row['oldpeak']}, slope of the peak exercise ST segment {row['slope']}, "
                       f"number of major vessels {row['ca']}, thalassemia status {row['thal']}, and target diagnosis {row['target']}.")
        descriptions.append(description)
    return descriptions

In [5]:
# Get the text based data
text_data = create_descriptions(df)

### Initiate the model

In [6]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

d:\src_git\learn_ds\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

# Correct paths for model and tokenizer
model_path = "D:\\src_git\\models\\Llama-3.2-1B"
tokenizer_path = "D:\\src_git\\models\\Llama-3.2-1B\\original"

# Load the tokenizer and model with the correct paths
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
model = LlamaForCausalLM.from_pretrained(model_path, trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [8]:
import sentencepiece as spm
print(spm.__version__)  # This should print the version if installed correctly


0.2.0


### Prepare the iteration function

In [9]:
from transformers import LlamaTokenizer

# Reinitialize tokenizer
tokenizer = LlamaTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [10]:
print(type(tokenizer))  # Should show <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>


<class 'bool'>


In [11]:
def ask_model(query):
    # Tokenize the input query
    inputs = tokenizer.encode(query, return_tensors="pt").to(device)
    # Generate response
    outputs = model.generate(inputs, max_length=100)
    # Decode the output
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer


### Query the model

In [12]:
# Sample question
question = "What are the common risk factors for heart disease?"
response = ask_model(question)
print(response)


AttributeError: 'bool' object has no attribute 'encode'